# Cryptocurrency Energy Derivatives Pricing Framework
## Interactive Real-Time Analysis with American Option Pricing

---

### Executive Summary

This notebook presents a **real-time derivatives pricing framework** for cryptocurrency energy costs using American option pricing theory. The framework:

- ✅ **Fetches live Bitcoin data** from CoinGecko API with real-time price, market cap, and volatility
- ✅ **Prices American options** using binomial tree models with early exercise features
- ✅ **Computes full Greeks** (Delta, Gamma, Vega, Theta, Rho) via finite difference methods
- ✅ **Interactive controls** with real-time parameter sliders and instant recalculation
- ✅ **Professional visualizations** including 3D surfaces, heatmaps, and multi-panel analysis

**Key Innovation**: We model cryptocurrency energy consumption as an underlying asset and price derivatives on this energy unit price, allowing for systematic risk management of Bitcoin's energy costs.

---

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from pricer import AmericanOptionPricer
from live_data import LiveDataFetcher
from visualizations import ProfessionalVisualizer

# Configure plotting
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

print("✓ All modules loaded successfully!")
print("✓ Ready for real-time analysis")

---

## Part 1: Real-Time Data Acquisition

We fetch **live Bitcoin market data** from the CoinGecko API, including:
- Current price and 24h change
- Market capitalization
- Trading volume
- Historical price data for volatility estimation
- Energy consumption estimates based on price

In [ ]:
# Initialize live data fetcher
print("Fetching live Bitcoin data from CoinGecko API...\n")
print("=" * 70)

fetcher = LiveDataFetcher()
live_params = fetcher.get_live_pricing_parameters(historical_days=365)

print("\n" + "=" * 70)
print("LIVE MARKET DATA")
print("=" * 70)
print(f"\n📊 Bitcoin Price:       ${live_params['btc_price']:,.2f}")
print(f"📈 Market Cap:          ${live_params['market_cap']/1e9:.1f}B")
print(f"⚡ Energy (estimated):  {live_params['energy_twh']:.1f} TWh/year")
print(f"\n🎯 Energy Unit Price (S₀): ${live_params['S0']:.4f}")
print(f"📉 Annualized Volatility:  {live_params['sigma']:.1%}")
print(f"⏰ Last Update:            {live_params['timestamp']}")
print(f"🔗 Data Source:            {live_params['source']}")
print("\n" + "=" * 70)

---

## Part 2: American Option Pricing with Live Data

Using the live market data, we price an **American call option** on the energy unit price.

**American options** differ from European options by allowing **early exercise** at any time before maturity. This is priced using a binomial tree with backward induction.

In [ ]:
# Create American option pricer with live data
S0 = live_params['S0']
K = live_params['K']
T = 1.0  # 1 year to maturity
r = 0.05  # 5% risk-free rate
sigma = live_params['sigma']

pricer = AmericanOptionPricer(S0=S0, K=K, T=T, r=r, sigma=sigma, N=100)

# Compute option value and Greeks
option_value = pricer.price()
greeks = pricer.compute_greeks()
boundary = pricer.compute_exercise_boundary()

print("=" * 70)
print("AMERICAN CALL OPTION PRICING RESULTS")
print("=" * 70)
print(f"\nOption Parameters:")
print(f"  Spot Price (S₀):    ${S0:.4f}")
print(f"  Strike Price (K):   ${K:.4f}")
print(f"  Time to Maturity:   {T:.2f} years")
print(f"  Volatility (σ):     {sigma:.1%}")
print(f"  Risk-free Rate (r): {r:.1%}")

print(f"\n💰 American Call Option Value: ${option_value:.4f}")

print(f"\nThe Greeks:")
print(f"  Delta (Δ):   {greeks['delta']:.4f}  [price sensitivity to spot]")
print(f"  Gamma (Γ):   {greeks['gamma']:.4f}  [delta sensitivity to spot]")
print(f"  Vega (ν):    {greeks['vega']:.4f}  [price sensitivity to volatility]")
print(f"  Theta (Θ):   {greeks['theta']:.4f}  [time decay per year]")
print(f"  Rho (ρ):     {greeks['rho']:.4f}  [price sensitivity to rates]")
print("\n" + "=" * 70)

---

## Part 3: Interactive Parameter Controls

**Adjust parameters in real-time** and see immediate recalculation of option prices and Greeks.

Use the sliders below to explore how different parameters affect the option value.

In [ ]:
def interactive_pricing(spot_multiplier, strike_multiplier, time_to_maturity, volatility, risk_free_rate):
    """
    Interactive option pricing with real-time updates.
    """
    # Use live data as base
    S = live_params['S0'] * spot_multiplier
    K_adj = live_params['S0'] * strike_multiplier
    
    # Create pricer with adjusted parameters
    pricer_interactive = AmericanOptionPricer(
        S0=S, 
        K=K_adj, 
        T=time_to_maturity, 
        r=risk_free_rate, 
        sigma=volatility, 
        N=50  # Reduced for speed
    )
    
    # Compute values
    value = pricer_interactive.price()
    greeks_int = pricer_interactive.compute_greeks()
    
    # Moneyness
    moneyness = S / K_adj
    if moneyness > 1.05:
        status = "Deep ITM 🟢"
    elif moneyness > 1.0:
        status = "In-the-Money 🟢"
    elif moneyness > 0.95:
        status = "At-the-Money 🟡"
    else:
        status = "Out-of-the-Money 🔴"
    
    # Display results
    print("=" * 70)
    print("REAL-TIME OPTION PRICING")
    print("=" * 70)
    print(f"\nCurrent Parameters:")
    print(f"  Spot Price (S₀):    ${S:.4f}")
    print(f"  Strike Price (K):   ${K_adj:.4f}")
    print(f"  Moneyness (S/K):    {moneyness:.2%} [{status}]")
    print(f"  Time to Maturity:   {time_to_maturity:.2f} years")
    print(f"  Volatility (σ):     {volatility:.1%}")
    print(f"  Risk-free Rate (r): {risk_free_rate:.1%}")
    
    print(f"\n💰 Option Value: ${value:.4f}")
    print(f"   Intrinsic Value: ${max(S - K_adj, 0):.4f}")
    print(f"   Time Value:      ${value - max(S - K_adj, 0):.4f}")
    
    print(f"\nThe Greeks:")
    print(f"  Δ = {greeks_int['delta']:.4f}    Γ = {greeks_int['gamma']:.4f}")
    print(f"  ν = {greeks_int['vega']:.4f}    Θ = {greeks_int['theta']:.4f}    ρ = {greeks_int['rho']:.4f}")
    print("\n" + "=" * 70)

# Create interactive widgets
interact(
    interactive_pricing,
    spot_multiplier=widgets.FloatSlider(
        value=1.0, min=0.5, max=1.5, step=0.05,
        description='Spot (×base):',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px')
    ),
    strike_multiplier=widgets.FloatSlider(
        value=1.0, min=0.5, max=1.5, step=0.05,
        description='Strike (×base):',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px')
    ),
    time_to_maturity=widgets.FloatSlider(
        value=1.0, min=0.1, max=3.0, step=0.1,
        description='Maturity (years):',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px')
    ),
    volatility=widgets.FloatSlider(
        value=sigma, min=0.1, max=1.0, step=0.05,
        description='Volatility (σ):',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        readout_format='.1%'
    ),
    risk_free_rate=widgets.FloatSlider(
        value=0.05, min=0.0, max=0.15, step=0.01,
        description='Risk-free Rate:',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        readout_format='.1%'
    )
);

---

## Part 4: Professional Visualizations

### 4.1 Comprehensive Multi-Panel Analysis

This figure shows 9 different perspectives on the American option:

In [ ]:
# Generate comprehensive analysis figure
print("Generating comprehensive analysis (this may take 20-30 seconds)...\n")

viz = ProfessionalVisualizer()
fig = viz.plot_comprehensive_analysis(pricer, greeks, boundary)
plt.show()

print("\n✓ Analysis complete!")

### 4.2 3D Option Value Surface

Visualize how option value changes across **spot price** and **volatility** dimensions:

In [ ]:
print("Generating 3D surface plot (this may take 30-40 seconds)...\n")

fig_surface = viz.plot_option_value_surface(
    pricer_class=AmericanOptionPricer,
    S0=S0,
    K=K,
    T=T,
    r=r,
    sigma_range=(0.2, 0.8),
    S_range=(S0*0.5, S0*1.5)
)

plt.show()

print("\n✓ 3D surface complete!")

### 4.3 Greeks Sensitivity Heatmaps

Show how all Greeks vary across spot price and volatility ranges:

In [ ]:
print("Generating Greeks heatmaps (this may take 40-50 seconds)...\n")

fig_heatmaps = viz.plot_greeks_heatmap(
    pricer_class=AmericanOptionPricer,
    S0=S0,
    K=K,
    T=T,
    r=r,
    base_sigma=sigma
)

plt.show()

print("\n✓ Heatmaps complete!")

---

## Part 5: Risk Management Applications

### Delta Hedging Strategy

Using the computed Delta, we can construct a **delta-neutral portfolio**:

In [ ]:
print("=" * 70)
print("DELTA HEDGING STRATEGY")
print("=" * 70)

delta = greeks['delta']
gamma = greeks['gamma']
vega = greeks['vega']

# Portfolio: Long 1000 call options
num_options = 1000
total_delta = num_options * delta
total_gamma = num_options * gamma
total_vega = num_options * vega

# Hedge: Short underlying
hedge_units = -total_delta  # Short delta units of underlying

print(f"\nPosition:")
print(f"  Long {num_options:,} American call options")
print(f"  Option Value: ${option_value:.4f} each")
print(f"  Total Investment: ${num_options * option_value:,.2f}")

print(f"\nPortfolio Greeks:")
print(f"  Total Delta:  {total_delta:,.2f}")
print(f"  Total Gamma:  {total_gamma:,.2f}")
print(f"  Total Vega:   {total_vega:,.2f}")

print(f"\nDelta Hedge:")
print(f"  {'SHORT' if hedge_units < 0 else 'LONG'} {abs(hedge_units):,.2f} units of underlying")
print(f"  Hedge Value: ${abs(hedge_units) * S0:,.2f}")

print(f"\nHedged Portfolio:")
print(f"  Net Delta:  ~0.00 (delta-neutral ✓)")
print(f"  Remaining Gamma: {total_gamma:,.2f} (re-hedge as spot moves)")
print(f"  Vega Exposure: {total_vega:,.2f} (unhedged volatility risk)")

print(f"\nProfit/Loss for Small Moves (linearized):")
for dS in [-0.10, -0.05, 0.0, 0.05, 0.10]:
    # First-order approximation: dV ≈ Delta * dS + 0.5 * Gamma * dS^2
    dV_option = num_options * (delta * dS + 0.5 * gamma * dS**2)
    dV_hedge = hedge_units * dS
    dV_total = dV_option + dV_hedge
    print(f"  S₀ {dS:+.2f}: Option P&L = ${dV_option:+,.2f}, Hedge P&L = ${dV_hedge:+,.2f}, Net = ${dV_total:+,.2f}")

print("\n" + "=" * 70)

---

## Part 6: Early Exercise Analysis

One of the key features of American options is the **early exercise premium**.

In [ ]:
print("=" * 70)
print("EARLY EXERCISE ANALYSIS")
print("=" * 70)

# Compute European option value for comparison
from scipy.stats import norm

def european_call_bs(S, K, T, r, sigma):
    """Black-Scholes European call price."""
    if T <= 0:
        return max(S - K, 0)
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

european_value = european_call_bs(S0, K, T, r, sigma)
american_value = option_value
early_exercise_premium = american_value - european_value

print(f"\nOption Values:")
print(f"  European Call:  ${european_value:.4f}")
print(f"  American Call:  ${american_value:.4f}")
print(f"  Early Exercise Premium: ${early_exercise_premium:.4f} ({early_exercise_premium/european_value:.1%})")

intrinsic = max(S0 - K, 0)
time_value_euro = european_value - intrinsic
time_value_amer = american_value - intrinsic

print(f"\nValue Decomposition:")
print(f"  Intrinsic Value:      ${intrinsic:.4f}")
print(f"  Time Value (Euro):    ${time_value_euro:.4f}")
print(f"  Time Value (Amer):    ${time_value_amer:.4f}")

print(f"\nInterpretation:")
if early_exercise_premium > 0.001:
    print(f"  ✓ Early exercise has significant value (${early_exercise_premium:.4f})")
    print(f"  ✓ American feature matters - may be optimal to exercise early")
else:
    print(f"  ✓ Early exercise premium negligible (${early_exercise_premium:.4f})")
    print(f"  ✓ American ≈ European for these parameters")

# Show exercise boundary
valid_boundary = boundary[~np.isnan(boundary)]
if len(valid_boundary) > 0:
    print(f"\nExercise Boundary:")
    print(f"  Critical Price at T=0:   ${valid_boundary[0]:.4f}")
    print(f"  Critical Price at T={T}: ${valid_boundary[-1]:.4f}")
    print(f"  Current Spot:            ${S0:.4f}")
    if S0 > valid_boundary[0]:
        print(f"  → Spot ABOVE boundary: Consider early exercise 🟢")
    else:
        print(f"  → Spot BELOW boundary: Hold option 🟡")

print("\n" + "=" * 70)

---

## Conclusion & Key Takeaways

### Summary of Results

This framework demonstrates:

1. **Real-time data integration** - Live Bitcoin market data flows directly into derivative pricing
2. **Rigorous American option pricing** - Binomial tree with early exercise features
3. **Complete Greeks computation** - All sensitivities computed via finite differences
4. **Interactive analysis** - Real-time parameter adjustment and recalculation
5. **Professional visualizations** - Publication-quality 3D surfaces, heatmaps, and multi-panel figures
6. **Risk management applications** - Delta hedging, early exercise analysis

### Practical Applications

- **Energy derivatives trading** - Hedge Bitcoin mining operations against energy price volatility
- **Token design** - Price redemption options and backing guarantees for cryptocurrency tokens
- **Portfolio risk management** - Construct delta-neutral hedges for crypto energy exposure
- **Research** - Empirical validation of option pricing models on cryptocurrency energy costs

### Technical Innovations

- **Energy unit price as underlying** - Novel approach to modeling Bitcoin's energy consumption
- **Live API integration** - Real-time data pipeline from CoinGecko
- **Interactive controls** - Jupyter widgets for parameter exploration
- **Comprehensive visualization suite** - Multiple perspectives on option behavior

---

### References

1. Hull, J.C. (2018). *Options, Futures, and Other Derivatives* (10th ed.). Pearson.
2. Cox, J.C., Ross, S.A., & Rubinstein, M. (1979). "Option pricing: A simplified approach." *Journal of Financial Economics*, 7(3), 229-263.
3. Black, F., & Scholes, M. (1973). "The pricing of options and corporate liabilities." *Journal of Political Economy*, 81(3), 637-654.
4. CoinGecko API Documentation: https://www.coingecko.com/en/api
5. Cambridge Bitcoin Electricity Consumption Index: https://ccaf.io/cbeci

---

**Framework Version**: 1.0  
**Last Updated**: 2025  
**Author**: Derivative Securities Final Project  
**License**: MIT with Academic Attribution

---